In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from summarytools import dfSummary
import json
from germansentiment import SentimentModel

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
click_data = pd.read_csv('dataset/click_data.csv')
emotional_events = pd.read_csv('dataset/emotional_events.csv')
messages_data = pd.read_csv('dataset/messages_data.csv')
# task_types = pd.read_csv('dataset/task_types.csv')
user_information = pd.read_csv('dataset/user_information.csv')

In [ ]:
# Preprocessing Data Types
emotional_events['valence'] = emotional_events['valence'].replace('AO07', '7').astype(float)
emotional_events['arousal'] = emotional_events['arousal'].replace('AO07', '7').astype(float)

In [ ]:
# Filtering Data

ee = emotional_events[emotional_events['task_type'] == 'information_finding']
len(ee)

# Create Emotion Quartiles

In [ ]:
ee['arousal'].unique()

In [ ]:
ee['valence'].unique()

In [ ]:
ee['valence_high'] = np.where(ee['valence'] > 3, 1, 0)
ee['arousal_high'] = np.where(ee['arousal'] > 3, 1, 0)

x: valence
y: arousal

|2|4|
|---|---|
|1|3|

In [ ]:
def get_emotion_score(df):
    if df['valence_high'] == 1 and df['arousal_high'] == 1:
        return 4
    elif df['valence_high'] == 1 and df['arousal_high'] == 0:
        return 3
    elif df['valence_high'] == 0 and df['arousal_high'] == 1:
        return 2
    elif df['valence_high'] == 0 and df['arousal_high'] == 0:
        return 1
    else:
        return np.nan
    
ee['emotion_quartile'] = ee.apply(get_emotion_score, axis=1)
ee.emotion_quartile.value_counts()

# Erstellung der neuen Features

In [ ]:
ee.info()

In [ ]:
messages_data.info()

In [ ]:
messages_data['agent_output'] = messages_data['input'].apply(lambda x: json.loads(x).get('output'))
messages_data['user_input'] = messages_data['input'].apply(lambda x: json.loads(x).get('input'))
messages_data

In [ ]:
ee.drop(columns='id', inplace=True)
ee.info()

## No answer given 

In [ ]:
# conversation ended in a technical error
df = pd.merge(ee, messages_data, left_on=['userId', 'task'], right_on=['userId', 'task'], how='left')

In [ ]:
df['no_answer_given'] = df['agent_output'] == "Leider habe ich keine Antwort auf diese Frage. Versuchen Sie bitte Ihre Frage anders zu formulieren, oder komplexe Fragestellungen in einfachere Fragen zu unterteilen."
print(df.no_answer_given.value_counts())

In [ ]:
ee

## Prompt Sentiment

In [ ]:
df.info()

In [ ]:
model = SentimentModel()
texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]
       
model.predict_sentiment(["Mit keinem guten Ergebniss"])

In [ ]:
df['input_y'].astype(str)

In [ ]:
unpacked_messages = pd.json_normalize(messages_data['input'].apply(json.loads).tolist()).values.flatten().tolist()

In [ ]:
df['sentiment'] = model.predict_sentiment(unpacked_messages)

In [ ]:
df

In [ ]:
cols_for_histplot = user_information.columns[4:]

In [ ]:
for col in cols_for_histplot:
    
    sns.histplot(user_information[col])
    plt.title(col)
    plt.savefig(f'{col}_histplot.png')
    plt.show()

In [ ]:
# dfSummary(messages_data)
# dfSummary(emotional_events)
# dfSummary(user_information)
# dfSummary(click_data)

In [ ]:
user_information['data_analysis_experience_bins'] = pd.cut(user_information['experience_analysis_tools'], bins=[0, 3, 5, 7], labels=['3 and lower', '4', '5 and higher'])
user_information['data_analysis_experience_bins'].value_counts()

In [ ]:
user_info_with_avg_valence = user_information.merge(emotional_events.groupby('userId')['valence'].mean().reset_index(), left_on='id', right_on='userId', how='left')
user_info_with_avg_valence

In [ ]:
sns.histplot(user_info_with_avg_valence[user_info_with_avg_valence['data_analysis_experience_bins'] == "5 and higher"]['valence'])
plt.title('Valence for users with 5 and higher experience')
plt.show()

In [ ]:
sns.histplot(user_info_with_avg_valence[user_info_with_avg_valence['data_analysis_experience_bins'] == "3 and lower"]['valence'])
plt.title('Valence for users with 3 and lower experience')
plt.show()

## Age


In [ ]:
user_information['age_bins'] = pd.cut(user_information['age'], bins=[0, 30, 50, 200], labels=['0-30', '30-50', 'older'])
user_info_with_avg_valence = user_information.merge(emotional_events.groupby('userId')['valence'].mean().reset_index(), left_on='id', right_on='userId', how='left')

In [ ]:
for bins in user_info_with_avg_valence['age_bins'].unique():
    subset = user_info_with_avg_valence[user_info_with_avg_valence['age_bins'] == bins]
    sns.histplot(subset['valence'])
    plt.title(f'Valence for users in {bins} age group')
    plt.savefig(f'valence_{bins}_age.png')
    plt.show()

## General Satisfaction

In [ ]:
user_information.columns

In [ ]:
user_information['satisfaction_bins'] = pd.cut(user_information['Satisfaction'], bins=[0, 3, 5, 7], labels=['3 and lower', '4', '5 and higher'])
user_info_with_avg_valence = user_information.merge(emotional_events.groupby('userId')['valence'].mean().reset_index(), left_on='id', right_on='userId', how='left')

In [ ]:
for bins in user_info_with_avg_valence['satisfaction_bins'].unique():
    subset = user_info_with_avg_valence[user_info_with_avg_valence['satisfaction_bins'] == bins]
    sns.histplot(subset['valence'])
    plt.title(f'Valence for users with {bins} satisfaction')
    plt.savefig(f'valence_{bins}_satisfaction.png')
    plt.show()

## Trust

In [ ]:
user_information['trust_bins'] = pd.cut(user_information['Trust 1'], bins=[0, 3, 5, 7], labels=['3 and lower', '4', '5 and higher'])
user_info_with_avg_valence = user_information.merge(emotional_events.groupby('userId')['valence'].mean().reset_index(), left_on='id', right_on='userId', how='left')

In [ ]:
for bins in user_info_with_avg_valence['trust_bins'].unique():
    subset = user_info_with_avg_valence[user_info_with_avg_valence['trust_bins'] == bins]
    sns.histplot(subset['valence'])
    plt.title(f'Valence for users with {bins} trust (1)')
    plt.savefig(f'valence_for_users_with_{bins}_trust.png')
    plt.show()

## education

In [ ]:
user_info_with_avg_valence = user_information.merge(emotional_events.groupby('userId')['valence'].mean().reset_index(), left_on='id', right_on='userId', how='left')


In [ ]:
for bins in user_info_with_avg_valence['education'].unique():
    subset = user_info_with_avg_valence[user_info_with_avg_valence['education'] == bins]
    sns.histplot(subset['valence'])
    plt.title(f'Valence for users with {bins} education')
    plt.savefig(f'Valence for users with {bins} education.png')
    plt.show()

## Mögliche Ideen
- wie wirkt sich der Assistent auf die Nutzer aus, je nach dem ob sie Vorerfahrung mit Datenanalyse haben oder nicht?
- 